In [3]:
import importlib

#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import utils
#import model
importlib.reload(utils)
#importlib.reload(model)
from utils import *
#from model import *

In [9]:

dic = get_dict_icon_sketches()

icons, icons_name_cat = load_icons(dic)
sketches, sketch_names_array = load_sketches(dic)

sketch_icon_indices, labels = create_positive_sketch_icon_indices(icons_name_cat, sketch_names_array)
print(f'We have {len(sketch_icon_indices)} samples.')
#TODO shuffle the indices

We have 31806 samples.


In [18]:
class Dataset():
    def __init__(self, indices):
        self.indices = indices

    def __len__(self):
        # Return the size of the dataset
        return len(self.indices)

    def __getitem__(self, index):
        # Retrieve inputs and targets at the given index
        indices = self.indices[index]

        return indices

def create_datasets(sketch_icon_indices, dataset_class, p_train=0.8, p_test=0.2):
    len_data = len(sketch_icon_indices)

    # Define partition sizes
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    sketch_icon_indices_Train = sketch_icon_indices[:num_train]
    sketch_icon_indices_Test = sketch_icon_indices[-num_test:]

    training_set = dataset_class(sketch_icon_indices_Train)
    test_set = dataset_class(sketch_icon_indices_Test)
    return training_set, test_set

training_set, test_set = create_datasets(sketch_icon_indices, Dataset)   
print(f'We have {len(training_set)} samples in the training set.')
print(f'We have {len(test_set)} samples in the test set.')

We have 25445 samples in the training set.
We have 6361 samples in the test set.


[0 0]
[25444   942]
[25445   942]
[31805  1177]
